In [3]:
%pip install librosa
%pip install pyaudio
%pip install SpeechRecognition

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
I have a model for real time emotion detection that uses video and audio data. The following is the code.
import tkinter as tk
from tkinter import filedialog
from tkinter import *
import cv2
from PIL import Image, ImageTk
import numpy as np
import speech_recognition
import librosa
import numpy as np
import wave


from tensorflow.keras.models import model_from_json

# Initialize Tkinter window
top = tk.Tk()
top.geometry('800x600')
top.title('Emotion Detector')
top.configure(background='#CDCDCD')

# Load the pre-trained model
def FacialExpressionModel(json_file, weights_file):
    with open(json_file, "r") as file:
        loaded_model_json = file.read()
        model = model_from_json(loaded_model_json)

    model.load_weights(weights_file)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Load pre-trained model and cascade classifier
facec = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
model = FacialExpressionModel("model_a1.json", "model_weights1.h5")

# Define emotions list
EMOTIONS_LIST = ["Angry", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]

# Function to extract MFCC features from audio
def extract_mfcc(y, sr):
    # Dummy function, replace with actual MFCC extraction code
    return None

def Detect():
    try:
        cap = cv2.VideoCapture(0)  # Use the default camera (usually webcam)
        sr = speech_recognition.Recognizer()  # Initialize the recognizer

        # Parameters for the WAV file
        n_channels = 1
        sample_width = 2  # in bytes
        frame_rate = 44100
        n_frames = 44100  # 1 second of audio

        # Open a new WAV file
        with wave.open('output.wav', 'w') as wav_file:
            wav_file.setnchannels(n_channels)
            wav_file.setsampwidth(sample_width)
            wav_file.setframerate(frame_rate)

            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = facec.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

                # Default value for pred_video
                pred_video = "No face detected"

                for (x, y, w, h) in faces:
                    fc = gray_frame[y:y + h, x:x + w]
                    roi = cv2.resize(fc, (48, 48))
                    pred_video = EMOTIONS_LIST[np.argmax(model.predict(roi[np.newaxis, :, :, np.newaxis]))]
                    cv2.putText(frame, pred_video, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

                cv2.imshow('Emotion Detector', frame)

                # Capture audio from microphone
                with speech_recognition.Microphone() as source:
                    print("Say something...")
                    audio = sr.listen(source)

                # Recognize speech and perform voice tone analysis
                try:
                    text = sr.recognize_google(audio)
                    print("Recognized text:", text)
                    # Perform voice tone analysis on the recognized text (this is just a placeholder)
                    # For example, you can analyze the sentiment of the text
                    # For simplicity, we'll assume the 'text' itself represents the emotion
                    pred_audio = text
                except speech_recognition.UnknownValueError:
                    pred_audio = "Unable to recognize"

                print("Predicted Emotion from Audio:", pred_audio)
                label1.configure(text="Video Emotion: " + pred_video + "\nAudio Emotion: " + pred_audio)

                # Write audio data to WAV file
                audio_data = np.frombuffer(audio.frame_data, dtype=np.int16)
                wav_file.writeframes(audio_data.tobytes())

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            cap.release()
            cv2.destroyAllWindows()
    except Exception as e:
        print("Error:", e)


# Create Detect button to start emotion detection
detect_button = Button(top, text="Detect Emotion", command=Detect, padx=10, pady=5)
detect_button.configure(background="#364156", foreground='white', font=('arial', 10, 'bold'))
detect_button.place(relx=0.79, rely=0.46)

# Label for displaying emotion
label1 = Label(top, background='#CDCDCD', font=('arial', 15, 'bold'))
label1.pack(side='bottom', expand='True')

# Title label
heading = Label(top, text='Emotion Detector', pady=20, font=('arial', 25, 'bold'))
heading.configure(background='#CDCDCD', foreground="#364156")
heading.pack()

top.mainloop()